# The goal for this program is to parse and anlyse the monthly weather data in Georgia given 7 key monitoring locations.

1.  Parse all 8 PDFs to extract daily precipitation (and temp if available).

2. Clean and merge the daily values by Date.

- For rainfall: sum across stations (total rain statewide).

- For temp: mean across stations (average state conditions).

🔁 Merge this with your March fungi dataset by date.

In [4]:
# %pip install pdfplumber

In [3]:

import pdfplumber
import pandas as pd

# Define a mapping of files to their respective locations
pdf_files = {
    "Monticello": "data/Climatological Data for MONTICELLO, GA - March 2025.pdf",
    "Dublin": "data/Climatological Data for DUBLIN 2, GA - March 2025.pdf",
    "Doraville": "data/Climatological Data for DORAVILLE 2 NNE, GA - March 2025.pdf",
    "Chatsworth": "data/Climatological Data for CHATSWORTH 2, GA - March 2025.pdf",
    "Carrollton": "data/Climatological Data for CARROLLTON, GA - March 2025.pdf",
    "Athens": "data/Climatological Data for ATHENS BEN EPPS AP, GA - March 2025.pdf",
    "Americus": "data/Climatological Data for AMERICUS, GA - March 2025.pdf"
}

# Function to extract text data from the first page of each PDF
extracted_data = {}

for location, path in pdf_files.items():
    with pdfplumber.open(path) as pdf:
        first_page = pdf.pages[0].extract_text()
        extracted_data[location] = first_page

# Display a sample for verification
{loc: txt[:600] for loc, txt in extracted_data.items()}  # preview first 600 chars of each location's text


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


{'Monticello': 'Climatological Data for MONTICELLO, GA - March 2025\nTemperature\nDate HDD CDD Precipitation New Snow Snow Depth\nMaximum Minimum Average Departure\n2025-03-01 72 26 49.0 -0.9 16 0 0.00 0.0 0\n2025-03-02 75 26 50.5 0.4 14 0 0.00 0.0 0\n2025-03-03 61 30 45.5 -4.9 19 0 0.00 0.0 0\n2025-03-04 63 30 46.5 -4.1 18 0 0.00 0.0 0\n2025-03-05 71 36 53.5 2.6 11 0 8.20 0.0 0\n2025-03-06 64 38 51.0 -0.1 14 0 0.00 0.0 0\n2025-03-07 57 32 44.5 -6.9 20 0 0.00 0.0 0\n2025-03-08 69 33 51.0 -0.6 14 0 0.00 0.0 0\n2025-03-09 71 51 61.0 9.1 4 0 0.00 0.0 0\n2025-03-10 56 44 50.0 -2.1 15 0 0.12 0.0 0\n2025-03-11 56 33 44.5 -7.9 20',
 'Dublin': 'Climatological Data for DUBLIN 2, GA - March 2025\nTemperature\nDate HDD CDD Precipitation\nMaximum Minimum Average Departure\n2025-03-01 75 42 58.5 5.9 6 0 0.00\n2025-03-02 80 41 60.5 7.7 4 0 0.00\n2025-03-03 58 28 43.0 -10.0 22 0 0.01\n2025-03-04 64 36 50.0 -3.2 15 0 0.00\n2025-03-05 76 47 61.5 8.0 3 0 0.68\n2025-03-06 72 41 56.5 2.8 8 0 0.00\n2025-03

# Parser code to extract, clean, and organize the following daily fields:

- Date
- Max Temp (if available)
- Min Temp (if available)
- Avg Temp (if available)
- Precipitation

In [8]:
import re

# Define a function to extract weather data from a raw text block
def extract_weather_data(text, location):
    lines = text.splitlines()
    data = []
    
    for line in lines:
        # Match lines starting with a valid date and containing at least precipitation
        match = re.match(r"(2025-03-\d{2})\s+([\dM.]+)?\s*([\dM.]+)?\s*([\dM.]+)?\s*[-\d.M]*\s*\d*\s*\d*\s*([\dM.]+)", line)
        if match:
            date = match.group(1)
            max_temp = match.group(2) if match.group(2) != 'M' else None
            min_temp = match.group(3) if match.group(3) != 'M' else None
            avg_temp = match.group(4) if match.group(4) != 'M' else None
            precip = match.group(5) if match.group(5) != 'M' else None

            data.append({
                "Date": date,
                "Location": location,
                "Max Temp": float(max_temp) if max_temp else None,
                "Min Temp": float(min_temp) if min_temp else None,
                "Avg Temp": float(avg_temp) if avg_temp else None,
                "Precipitation": float(precip) if precip else 0.0  # Treat missing precip as 0.0 for summing
            })
    return data

# Apply the function to all extracted text
all_weather_records = []
for location, raw_text in extracted_data.items():
    records = extract_weather_data(raw_text, location)
    all_weather_records.extend(records)

# Create DataFrame
weather_df = pd.DataFrame(all_weather_records)
weather_df['Date'] = pd.to_datetime(weather_df['Date'])

from IPython.display import display

# Display the DataFrame in a Jupyter-friendly format
display(weather_df)


,Date,Location,Max Temp,Min Temp,Avg Temp,Precipitation
0,2025-03-01,Monticello,72.00,26.0,49.0,0.0
1,2025-03-02,Monticello,75.00,26.0,50.5,0.0
2,2025-03-03,Monticello,61.00,30.0,45.5,0.0
3,2025-03-04,Monticello,63.00,30.0,46.5,0.0
4,2025-03-05,Monticello,71.00,36.0,53.5,8.2
...,...,...,...,...,...,...
212,2025-03-27,Americus,0.00,0.0,NaN,0.0
213,2025-03-28,Americus,0.00,0.0,NaN,0.0
214,2025-03-29,Americus,0.00,0.0,NaN,0.0
215,2025-03-30,Americus,0.00,0.0,NaN,0.0


# Aggregate daily values statewide (mean temp, total rainfall)

In [9]:
# Aggregate the weather data by Date
daily_aggregated_weather = weather_df.groupby("Date").agg({
    "Max Temp": "mean",
    "Min Temp": "mean",
    "Avg Temp": "mean",
    "Precipitation": "sum"
}).reset_index()

# Rename for clarity
daily_aggregated_weather.rename(columns={
    "Max Temp": "Avg Max Temp (°F)",
    "Min Temp": "Avg Min Temp (°F)",
    "Avg Temp": "Avg Temp (°F)",
    "Precipitation": "Total Precipitation (in)"
}, inplace=True)

display(daily_aggregated_weather)

,Date,Avg Max Temp (°F),Avg Min Temp (°F),Avg Temp (°F),Total Precipitation (in)
0,2025-03-01,50.142857,31.333333,53.90,0.00
1,2025-03-02,48.428571,28.166667,50.80,0.00
2,2025-03-03,41.285714,23.166667,42.80,1.01
3,2025-03-04,45.714286,26.166667,47.70,0.00
4,2025-03-05,46.646667,44.000000,56.75,12.40
5,2025-03-06,44.574286,32.333333,50.60,4.92
6,2025-03-07,42.142857,28.000000,46.30,0.00
7,2025-03-08,49.285714,31.666667,53.50,0.01
8,2025-03-09,48.587143,47.800000,57.90,0.03
9,2025-03-10,39.800000,43.400000,49.30,10.67


In [10]:
# Save weather_df to a CSV file
weather_df.to_csv('GA_weather_data_march2025.csv', index=False)

# Save daily_aggregated_weather to a CSV file
daily_aggregated_weather.to_csv('GA_daily_aggregated_weather_march25.csv', index=False)

# The previous method leads to incorrect data parsing and aggregation due to variation of pdf headers
- moving forward, data is acquired manually 
- we go through the aggregation process again


In [5]:
import pandas as pd
from IPython.display import display

# import the CSV file
new_weather_df = pd.read_csv('data/Weather Data Manual Entries - March-25.csv')

# Convert the 'Date' column to datetime format
new_weather_df['Date'] = pd.to_datetime(new_weather_df['Date'], format='%Y-%m-%d')

# Display the DataFrame in a Jupyter-friendly format
display(new_weather_df)

# aggregate the weather data by Date
v2_daily_aggregated_weather_manual = new_weather_df.groupby("Date").agg({
    "Avg Temp": "mean",
    "Precipitation": "sum"
}).reset_index()

# Display the aggregated DataFrame in a Jupyter-friendly format
display(v2_daily_aggregated_weather_manual)

# Save the aggregated DataFrame to a CSV file
v2_daily_aggregated_weather_manual.to_csv('data/v2_GA_daily_aggregated_weather_manual_march25.csv', index=False)

,Date,Location,Avg Temp,Precipitation
0,2025-03-01,Monticello,49.0,0.00
1,2025-03-02,Monticello,50.5,0.00
2,2025-03-03,Monticello,45.5,0.00
3,2025-03-04,Monticello,46.5,0.00
4,2025-03-05,Monticello,53.5,8.20
...,...,...,...,...
212,2025-03-27,Americus,NaN,0.00
213,2025-03-28,Americus,NaN,0.00
214,2025-03-29,Americus,NaN,0.00
215,2025-03-30,Americus,NaN,0.00


,Date,Avg Temp,Precipitation
0,2025-03-01,53.90,0.00
1,2025-03-02,50.80,0.00
2,2025-03-03,42.80,0.02
3,2025-03-04,47.70,0.01
4,2025-03-05,56.75,11.62
5,2025-03-06,50.60,1.66
6,2025-03-07,46.30,0.00
7,2025-03-08,53.50,0.00
8,2025-03-09,57.90,0.26
9,2025-03-10,49.30,5.27
